# Import de bibliotecas

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1 - Funções

In [ ]:
def remover_espacos(raw_data):
    df_obj = raw_data.select_dtypes(['object']) 
    raw_data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [ ]:
def filtrar_classes_de_interesse(classes_de_interesse, raw_data):
    
    dict_classes = dict(zip(raw_data.level_4.unique(), raw_data.class_id.unique()))
    lista_filtrados = []
    
    for i in classes_de_interesse:
        lista_filtrados.append(raw_data.loc[(raw_data.class_id == dict_classes[i]) & (raw_data.UF == 'PA')])

    dados_filtrados = pd.concat(lista_filtrados)[['municipality', 'class_id','level_4', '1985', '1986',
                                                '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
                                                '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
                                                '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
                                                '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']].fillna(0)

    return dados_filtrados

In [ ]:
def carregar_arquivos_base(download, links, paths):
    
    if download:
        try: 
            pd.read_excel(links[0], #Extraindo dados da fonte (excel)
                    sheet_name='COBERTURA_COL7').to_csv(paths[0],
                    index=False) # Salvando dados em .CSV

            pd.read_excel(links[1], #Extraindo dados da fonte (excel)
                    sheet_name='Planilha1').to_csv(paths[1], 
                    index=False) # Salvando dados em .CSV
        except:
            raise BrokenPipeError('Erro de extração dos dados do mapbiomas ou divisão regional')

    try:
        cobertura_terra = pd.read_csv(paths[0]) # Carregando dados de uso da terra
        micro_reg = pd.read_csv(paths[1]).rename(columns={'nome_mun':'municipality', 'CD_GEOCODI':'cod_municipality'}) # Carregando dados de microregiões
        
    except:
        raise FileNotFoundError('Dados de cobertura da terra não encontrados')

    return cobertura_terra, micro_reg

In [17]:
def gerar_titulo(texto, simbolo):
    num_simbolos = (65 - len(texto)) / 2
    print(simbolo * int(num_simbolos), texto, simbolo * int(num_simbolos))

In [ ]:
def merge_dados_cobertura(dados, micro_reg):
    dados_merge = dados.merge(micro_reg, how = 'inner', on='municipality') # Mesclagem de dados
    dados_merge = dados_merge.rename(columns = {'municipality':'municipio', 'class_id':'id_classe',
                                                'nome_rgint':'regiao_intermediaria', 'cod_rgint':'cod_regiao_intermediaria',
                                                'nome_rgi':'regiao_imediata',
                                                'cod_rgi':'cod_regiao_imediata',
                                                'level_4':'nome_classe'})
    return dados_merge

In [32]:
def download_data_SIDRA(download):
    
    if download:
        lavouras_temporarias = {"https://apisidra.ibge.gov.br/values/t/1612/n6/in%20n3%2015/p/all/v/214,109,216/c81/":[2688, 2691, 2692,2694,2696,2697,2702,2703,2706,2706,2709,2711,2713,2714,2715]}
        produto_das_sivilcultura ={'https://apisidra.ibge.gov.br/values/t/291/n6/in%20n3%2015/p/all/v/142/c194/':[33247,33250,33253,33256,33258]}
        produto_das_pecuaria = {"https://apisidra.ibge.gov.br/values/t/3939/n6/in%20n3%2015/p/all/v/105/c79/":[2670, 2675, 2672, 2681, 2677]}
        produto_das_lavouras_permanentes = {"https://apisidra.ibge.gov.br/values/t/1613/n6/in%20n3%2015/p/all/v/216,2313,214/c82/":[45981,2720,2721,2722,2723,31619,2725,2727,2731,2733,2734,2736,2738,90001,2743,2745,2747]}


        tabelas_sidra = {
                        '1612':lavouras_temporarias,
                        '291':produto_das_sivilcultura,
                        '3939':produto_das_pecuaria,
                        '1613':produto_das_lavouras_permanentes,
                        }


        for k,v in tabelas_sidra.items():
            a = []
            for k2, v2 in v:
                link = k2 + f"{v2}/h/n"
                try:
                    print(f'tentando: {v2}')
                    df = pd.read_json(link)
                    print(f'concluido: {v2}')
                    a.append(df)
                except:
                    print(f'falha: {v2}')


            table = pd.concat(a)

            if k == '1612':
                table['tipo_producao'] = 'Produto das lavouras permanentes'
            elif k == '1613':
                table['tipo_producao'] = "Produto das lavouras temporárias"
            elif k == '291':
                table['tipo_producao'] = 'Silvicultura'
            elif k == '3939':
                table['tipo_producao'] = 'Pecuária'
            
            

            table_columns_names = ['cod_mun', 'mun', 'cod_metrica', 'metrica', 
                                    'valor', 'cod_cidade', 'cidade', 'cod_ano', 
                                    'ano', 'cod_variavel', 'variavel', 'cod_produto', 
                                    'produto', 'tipo_producao']

            table.columns = table_columns_names
            table.to_csv(f"/content/drive/MyDrive/Dados/Dados_SIDRA/table_{k}.csv", index= False)
    else:
        print('SIDRA download skipped')
    

In [35]:
def name_class_cobertura(cobertura_long):
    for index, row in cobertura_long.iterrows():

        if row['nome_classe'] == 'Forest Formation':
            cobertura_long.loc[index,'variavel'] = 'Área de cobertura'
            cobertura_long.loc[index,'produto'] = 'Florest'
            cobertura_long.loc[index,'tipo_producao'] = 'Área de cobertura de Floresta'
        elif row['nome_classe'] == 'Pasture':
            cobertura_long.loc[index,'variavel'] = 'Área de cobertura'
            cobertura_long.loc[index,'produto'] = 'Pasto'
            cobertura_long.loc[index,'tipo_producao'] = 'Pastagem'
        elif row['nome_classe'] == 'Soy Beans':
            cobertura_long.loc[index,'variavel'] = 'Área de cobertura'
            cobertura_long.loc[index,'produto'] = 'Soja (em grão)'
            cobertura_long.loc[index,'tipo_producao'] = 'Produto das lavouras temporárias'
        elif row['nome_classe'] == 'Other Temporary Crops':
            cobertura_long.loc[index,'variavel'] = 'Área de cobertura'
            cobertura_long.loc[index,'produto'] = 'Outros'
            cobertura_long.loc[index,'tipo_producao'] = 'Produto das lavouras temporárias'
        elif row['nome_classe'] == 'Other Perennial Crops':
            cobertura_long.loc[index,'variavel'] = 'Área de cobertura'
            cobertura_long.loc[index,'produto'] = 'Outros'
            cobertura_long.loc[index,'tipo_producao'] = 'Produto das lavouras permanentes'
        elif row['nome_classe'] == 'Forest Plantation':
            cobertura_long.loc[index,'variavel'] = 'Área de cobertura'
            cobertura_long.loc[index,'produto'] = 'Silvicultura'
            cobertura_long.loc[index,'tipo_producao'] = 'Silvicultura'

    cobertura_long['metrica'] = 'Hectares'
    return cobertura_long

Pendente correção total

In [34]:
# União das tabelas da SIDRA
def load_sidra_unified():
    lavouras_temporarias = pd.read_csv('/content/drive/MyDrive/Dados/Dados_SIDRA/table_1612-lavouras_temporarias.csv') # Table of temporary crops
    lavouras_permanentes = pd.read_csv('/content/drive/MyDrive/Dados/Dados_SIDRA/table_1613-lavouras_permanentes.csv') # Table of permanents crops
    sivilcultura = pd.read_csv('/content/drive/MyDrive/Dados/Dados_SIDRA/table_291-sivilcultura.csv')                  # Table of florestry
    pecuaria = pd.read_csv('/content/drive/MyDrive/Dados/Dados_SIDRA/table_3939-pecuaria.csv')                         # Livestocks

    micro_regioes = pd.read_csv('/content/drive/MyDrive/Dados/dados-de-apoio/Microregioes.csv')

    # Changing column name for cities code (ID)
    micro_regioes = micro_regioes.rename(columns = {'CD_GEOCODI':'cod_cidade'})
    micro_regioes.head()

    iter_tables = [pecuaria, sivilcultura, lavouras_temporarias, lavouras_permanentes]
    column_names = lavouras_permanentes.columns
    sidra_list = []

    for i in iter_tables:
        i.columns = column_names
        sidra_list.append(i)

        # Unifying tables
    sidra_df = pd.concat(sidra_list)

    # Reseting index 
    sidra_df = sidra_df.reset_index(drop = True)

    # Formating cities names (Removing suffix) 
    sidra_df.cidade = sidra_df.cidade.str.replace(' - PA', '')

    # Strip on cities names
    sidra_df['cidade'] = sidra_df.cidade.str.strip()
   

    # Merging tables of the unyfied table and micro regions table
    sidra_df = sidra_df.merge(micro_regioes, how = 'inner', on = 'cod_cidade')

    # Changing the order of final table with the purpose of formating
    sidra_df = sidra_df[['cidade', 'cod_cidade', 'nome_rgi', 'cod_rgi', 'cod_rgint', 'nome_rgint', 
                        'tipo_producao', 'cod_variavel', 'variavel', 'cod_produto', 'produto',
                        'cod_metrica','metrica', 'valor', 'ano']]

    # Saving final table formated with name "table_full"
    sidra_df.to_csv('/content/drive/MyDrive/Dados/Dados_SIDRA/table_full.csv', index = False)

In [ ]:
def to_long_cobertura(df_cobertura, df_sifra):
  
    cobertura_long = pd.melt(df_cobertura, id_vars = df_cobertura.iloc[:,:8].columns, value_vars = df_cobertura.iloc[:,8:].columns)
    name_class_cobertura(cobertura_long)

    cobertura_long.columns = ['municipio', 'cod_municipality','cod_regiao_intermediaria',
                          'regiao_intermediaria','cod_regiao_imediata',	'regiao_imediata','cod_variavel',
                          'nome_classe','ano','valor','variavel','produto','tipo_producao','metrica']
    cobertura_long = cobertura_long.drop(columns = 'nome_classe')

    df_sifra = df_sifra.drop(columns = ['cod_produto', 'cod_metrica'])  
    df_sifra.columns = ['municipio', 'cod_municipality', 'regiao_intermediaria',
                    'cod_regiao_intermediaria', 'cod_regiao_imediata', 'regiao_imediata',
                    'tipo_producao', 'cod_variavel', 'variavel', 'produto','metrica', 'valor', 'ano']


    cobertura_long = cobertura_long[df_sifra.columns]

    unified_df = pd.concat([df_sifra, cobertura_long])

    unified_df.to_csv('/content/drive/MyDrive/Dados/unified-data/full_table.csv', index=False)



# 2 - PIPELINE

In [ ]:
# =================================================================
# ==================== Definição de parâmetros ====================
# =================================================================

# ----------------------- Classe de interesse ---------------------
classes_de_interesse = ['Forest Formation', 'Pasture', 
                        'Soy Beans', 'Other Temporary Crops',
                        'Forest Plantation', 'Other Perennial Crops']

# ------------------------------ Links ----------------------------

link_mapbiomas = 'https://storage.googleapis.com/mapbiomas-public/brasil/downloads/1-tabela-geral-col7-mapbiomas-biomas-municipio-final.xlsx'
link_divisao_regional = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/divisao_regional/divisao_regional_do_brasil/divisao_regional_do_brasil_em_regioes_geograficas_2017/tabelas/regioes_geograficas_composicao_por_municipios_2017_20180911.xlsx'

links = [link_mapbiomas, link_divisao_regional]


# ---------------------------- Caminhos ----------------------------

path_cobertura_terra = "/content/drive/MyDrive/Dados/DW_Cobertura-da-terra/Cobertura.csv"
path_microreg = '/content/drive/MyDrive/Dados/dados-de-apoio/Microregioes.csv'

paths = [path_cobertura_terra, path_microreg]

# *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-





# =================================================================
# ======================= Load arquivos base ======================
# =================================================================

cobertura_terra, micro_reg = carregar_arquivos_base(False, links, paths)



# =================================================================
# ============================ PIPELINE ===========================
# =================================================================


# ----------------------- COBERTURA DA TERRA ----------------------

raw_data = remover_espacos(cobertura_terra)
raw_data = filtrar_classes_de_interesse(classes_de_interesse, cobertura_terra)
raw_data = merge_dados_cobertura(raw_data, micro_reg)

# ------------------------------ SIDRA ------------------------------
download_data_SIDRA(False) # modificar para carregar os dados

